# Amazon SageMaker로 다중 노드들 간 분산 RL을 이용해 Roboschool 에이전트 훈련
---
이 노트북은 `rl_roboschool_ray.ipynb` 의 확장으로, Ray와 TensorFlow를 사용한 강화 학습의 수평(horizontal) 스케일링을 보여줍니다.

## 해결해야 할 Roboschool 문제 선택

Roboschool은 가상 로봇 시스템에 대한 RL 정책을 훈련시키는 데 주로 사용되는 [오픈 소스](https://github.com/openai/roboschool/tree/master/roboschool) 물리 시뮬레이터입니다. Roboschool은 다양한 로봇 문제에 해당하는 [다양한](https://github.com/openai/roboschool/blob/master/roboschool/__init__.py) gym 환경을 정의합니다. 아래는 다양한 난이도 중 몇 가지를 보여줍니다.

- **Reacher (쉬움)** - 2개의 조인트만 있는 매우 간단한 로봇이 목표물에 도달합니다.
- **호퍼 (중간)** - 한쪽 다리와 발이 달린 간단한 로봇이 트랙을 뛰어 내리는 법을 배웁니다.
- **휴머노이드 (어려움)** - 두 개의 팔, 두 개의 다리 등이 있는 복잡한 3D 로봇은 넘어지지 않고 균형을 잡은 다음 트랙에서 달리는 법을 배웁니다.

간단한 문제들은 적은 계산 리소스 상에서 더 빨리 훈련됩니다. 물론 더 복잡한 문제들은 훈련이 느리지만 더 재미있습니다.


In [ ]:
# Uncomment the problem to work on
#roboschool_problem = 'reacher'
#roboschool_problem = 'hopper'
roboschool_problem = 'humanoid'

## 전제 조건(Pre-requisites)

### 라이브러리 임포트

시작하기 위해, 필요한 Python 라이브러리를 가져와서 권한 및 구성을 위한 몇 가지 전제 조건으로 환경을 설정합니다.

In [ ]:
import sagemaker
import boto3
import sys
import os
import glob
import re
import subprocess
from IPython.display import HTML, Markdown
import time
from time import gmtime, strftime
sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from docker_utils import build_and_push_docker_image
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework
from markdown_helper import generate_help_for_s3_endpoint_permissions, create_s3_endpoint_manually

### S3 버킷 설정

체크포인트(checkpoint) 및 메타데이터에 사용하려는 S3 버킷에 대한 연결 및 인증을 설정합니다.

In [ ]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

### 변수 설정

훈련 작업의 작업 접두사(job prefix)와 *컨테이너의 이미지 경로(BYOC 인 경우에만)와 같은 변수*를 정의합니다.

In [ ]:
# create a descriptive job name 
job_name_prefix = 'rl-roboschool-distributed-' + roboschool_problem
aws_region = boto3.Session().region_name

### 훈련이 진행되는 위치 구성

SageMaker 노트북 인스턴스 또는 로컬 노트북 인스턴스를 사용하여 RL 훈련 작업을 훈련할 수 있습니다. 로컬 모드는 SageMaker Python SDK를 사용하여 SageMaker에 배포하기 전에 로컬 컨테이너에서 코드를 실행합니다. 이렇게 하면 , 익숙한 Python SDK 인터페이스를 사용하면서 반복 테스트 및 디버깅 속도를 높일 수 있습니다. 여러분은 `local_mode = True` 만 설정하면 됩니다.

In [ ]:
# run in local_mode on this machine, or as a SageMaker TrainingJob?
local_mode = False

if local_mode:
    instance_type = 'local'
else:
    # If on SageMaker, pick the instance type
    instance_type = "ml.c5.2xlarge"
    
train_instance_count = 3

### IAM 역할 생성

SageMaker 노트북 `role = sagemaker.get_execution_role()`을 실행할 때 실행 역할(execution role)을 얻거나 로컬 시스템에서 실행할 때 utils 메소드 `role = get_execution_role()`을 사용하여 실행 역할을 작성하세요.

In [ ]:
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

### `로컬` 모드용 도커 설치

로컬 모드에서 작업하려면 도커(docker)가 설치되어 있어야 합니다. 로컬 머신에서 실행할 때는 docker 또는 docker-compose(로컬 CPU 머신의 경우) 및 nvidia-docker(로컬 GPU 머신의 경우)가 설치되어 있는지 확인하세요. 또는, SageMaker 노트북 인스턴스에서 실행할 때 다음 스크립트를 실행하여 관련 패키지들을 설치할 수 있습니다.

참고로, 한 번에 하나의 로컬 노트북만 실행할 수 있습니다.

In [ ]:
# only run from SageMaker notebook instance
if local_mode:
    !/bin/bash ./common/setup.sh

## 도커 컨테이너 빌드

Roboschool이 설치된 사용자 정의 도커 컨테이너를 빌드해야 합니다. 컨테이너 빌드 작업은 아래 과정을 거쳐 처리됩니다.

1. 기본 컨테이너 이미지 가져오기
2. Roboschool 및 의존성 패키지 설치
3. 새 컨테이너 이미지를 ECR에 업로드

인터넷 연결이 느린 컴퓨터에서 실행 중인 경우, 이 단계에서 시간이 오래 걸릴 수 있습니다. 노트북 인스턴스가 SageMaker 또는 EC2 인 경우 인스턴스 유형에 따라 3-10 분이 걸립니다.

In [ ]:
%%time

cpu_or_gpu = 'gpu' if instance_type.startswith('ml.p') else 'cpu'
repository_short_name = "sagemaker-roboschool-ray-%s" % cpu_or_gpu
docker_build_args = {
    'CPU_OR_GPU': cpu_or_gpu, 
    'AWS_REGION': boto3.Session().region_name,
}
custom_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
print("Using ECR image %s" % custom_image_name)

## 훈련 코드 작성

훈련 코드는 `/src` 디렉토리에 업로드된 `“train-{roboschool_problem}.py”` 파일에 작성됩니다. 먼저 환경 파일과 사전 설정 파일을 가져온 다음, `main()` 함수를 정의하세요.

In [ ]:
!pygmentize src/train-{roboschool_problem}.py

## Ray 동종 스케일링 - train_instance_count > 1 지정

동종(Homogeneous) 스케일링을 통해 동일한 유형의 여러 인스턴스를 사용할 수 있습니다.

In [ ]:
metric_definitions = RLEstimator.default_metric_definitions(RLToolkit.RAY)
    
estimator = RLEstimator(entry_point="train-%s.py" % roboschool_problem,
                        source_dir='src',
                        dependencies=["common/sagemaker_rl"],
                        image_name=custom_image_name,
                        role=role,
                        train_instance_type=instance_type,
                        train_instance_count=train_instance_count,
                        output_path=s3_output_path,
                        base_job_name=job_name_prefix,
                        metric_definitions=metric_definitions,
                        hyperparameters={
                          # Attention scientists!  You can override any Ray algorithm parameter here:
                            
                            # 3 m4.2xl with 8 cores each. We have to leave 1 core for ray scheduler.
                            # Don't forget to change this on the basis of instance type.
                            "rl.training.config.num_workers": (8 * train_instance_count) - 1
                            
                          #"rl.training.config.horizon": 5000,
                          #"rl.training.config.num_sgd_iter": 10,
                        }
                    )

estimator.fit(wait=local_mode)
job_name = estimator.latest_training_job.job_name
print("Training job: %s" % job_name)

## 시각화

RL 훈련에는 시간이 오래 걸릴 수 있습니다. 따라서 훈련 작업이 동작하는 동안 훈련 작업의 진행 상황을 추적할 수 있는 다양한 방법들이 있습니다. 훈련 도중 일부 중간 출력이 S3에 저장되므로, 이를 캡처하도록 설정합니다.

In [ ]:
print("Job name: {}".format(job_name))

s3_url = "s3://{}/{}".format(s3_bucket,job_name)

if local_mode:
    output_tar_key = "{}/output.tar.gz".format(job_name)
else:
    output_tar_key = "{}/output/output.tar.gz".format(job_name)

intermediate_folder_key = "{}/output/intermediate/".format(job_name)
output_url = "s3://{}/{}".format(s3_bucket, output_tar_key)
intermediate_url = "s3://{}/{}".format(s3_bucket, intermediate_folder_key)

print("S3 job path: {}".format(s3_url))
print("Output.tar.gz location: {}".format(output_url))
print("Intermediate folder path: {}".format(intermediate_url))
    
tmp_dir = "/tmp/{}".format(job_name)
os.system("mkdir {}".format(tmp_dir))
print("Create local folder {}".format(tmp_dir))

### 훈련 롤아웃 비디오 가져오기

특정 롤아웃의 비디오는 훈련 중 S3에 기록됩니다. 여기에서는 S3에서 마지막 10개의 비디오 클립을 가져 와서 마지막 비디오를 렌더링합니다.

In [ ]:
recent_videos = wait_for_s3_object(s3_bucket, intermediate_folder_key, tmp_dir, 
                                fetch_only=(lambda obj: obj.key.endswith(".mp4") and obj.size>0), limit=10)

In [ ]:
last_video = sorted(recent_videos)[-1]  # Pick which video to watch
os.system("mkdir -p ./src/tmp_render_homogeneous/ && cp {} ./src/tmp_render_homogeneous/last_video.mp4".format(last_video))
HTML('<video src="./src/tmp_render_homogeneous/last_video.mp4" controls autoplay></video>')

### 훈련 작업에 대한 지표 plot

CloudWatch 지표에 기록된 알고리즘 지표를 사용하여 실행 중인 훈련의 보상 지표를 볼 수 있습니다. 시간이 지남에 따라, 모델의 성능을 볼 수 있도록 이를 plot할 수 있습니다.

In [ ]:
%matplotlib inline
from sagemaker.analytics import TrainingJobAnalytics

df = TrainingJobAnalytics(job_name, ['episode_reward_mean']).dataframe()
num_metrics = len(df)
if num_metrics == 0:
    print("No algorithm metrics found in CloudWatch")
else:
    plt = df.plot(x='timestamp', y='value', figsize=(12,5), legend=True, style='b-')
    plt.set_ylabel('Mean reward per episode')
    plt.set_xlabel('Training time (s)')

### 훈련 진행 상황 모니터링

위의 시각화 셀을 반복해서 실행하여 최신 비디오를 얻거나, 훈련 작업이 진행됨에 따라 최신 지표를 볼 수 있습니다.

## Ray 이기종(heterogeneous) 스케일링

RL 훈련을 확장하기 위해 롤아웃 작업자 수를 늘릴 수 있습니다. 그러나, 롤아웃이 많을수록 훈련 중 종종 병목 현상이 발생할 수 있습니다. 이를 방지하기 위해 하나 이상의 GPU가 있는 인스턴스를 훈련용으로 사용하고 여러 개의 CPU 인스턴스들을 롤아웃에 사용할 수 있습니다.

SageMaker는 훈련 작업에서 단일 유형의 인스턴스를 지원하므로, 두 개의 SageMaker 작업을 서로 통신하도록 함으로써 위의 목표를 달성할 수 있습니다. 이름 지정을 위해 `기본 클러스터(Primary cluster)`를 사용하여 하나 이상의 GPU 인스턴스를 참조하고 `보조 클러스터(Secondary cluster)`를 사용하여 CPU 인스턴스 클러스터를 참조합니다.

> local_mode는 이 유형의 스케일링을 테스트하는 데 사용할 수 없습니다.

SageMaker 작업을 구성하기 전에 먼저 VPC 모드에서 SageMaker를 실행해야 합니다. VPC 모드에서는 두 SageMaker 작업이 네트워크를 통해 통신할 수 있습니다.

작업 시작 스크립트에 서브넷(subnet)과 보안 그룹(security group)을 제공하면 됩니다. 이 예에서는 기본 VPC 구성을 사용합니다.

In [ ]:
ec2 = boto3.client('ec2')
default_vpc = [vpc['VpcId'] for vpc in ec2.describe_vpcs()['Vpcs'] if vpc["IsDefault"] == True][0]

default_security_groups = [group["GroupId"] for group in ec2.describe_security_groups()['SecurityGroups'] \
                   if group["GroupName"] == "default" and group["VpcId"] == default_vpc]

default_subnets = [subnet["SubnetId"] for subnet in ec2.describe_subnets()["Subnets"] \
                  if subnet["VpcId"] == default_vpc and subnet['DefaultForAz']==True]

print("Using default VPC:", default_vpc)
print("Using default security group:", default_security_groups)
print("Using default subnets:", default_subnets)

VPC 모드에서 실행 중인 SageMaker 작업은 S3 리소스에 액세스할 수 없습니다. 따라서, SageMaker 컨테이너에서 S3에 액세스할 수 있도록 VPC S3 엔드포인트를 생성해야 합니다. VPC 모드에 대한 자세한 내용을 보려면 [이 링크](https://docs.aws.amazon.com/sagemaker/latest/dg/train-vpc.html)를 방문하세요.

In [ ]:
try:
    route_tables = [route_table["RouteTableId"] for route_table in ec2.describe_route_tables()['RouteTables']\
                if route_table['VpcId'] == default_vpc]
except Exception as e:
    if "UnauthorizedOperation" in str(e):
        display(Markdown(generate_help_for_s3_endpoint_permissions(role)))
    else:
        display(Markdown(create_s3_endpoint_manually(aws_region, default_vpc)))
    raise e

print("Trying to attach S3 endpoints to the following route tables:", route_tables)

assert len(route_tables) >= 1, "No route tables were found. Please follow the VPC S3 endpoint creation "\
                              "guide by clicking the above link."

try:
    ec2.create_vpc_endpoint(DryRun=False,
                           VpcEndpointType="Gateway",
                           VpcId=default_vpc,
                           ServiceName="com.amazonaws.{}.s3".format(aws_region),
                           RouteTableIds=route_tables)
    print("S3 endpoint created successfully!")
except Exception as e:
    if "RouteAlreadyExists" in str(e):
        print("S3 endpoint already exists.")
    elif "UnauthorizedOperation" in str(e):
        display(Markdown(generate_help_for_s3_endpoint_permissions(role)))
        raise e
    else:
        display(Markdown(create_s3_endpoint_manually(aws_region, default_vpc)))
        raise e

### 인스턴스 유형 구성

1 Volta (V100) GPU와 40개의 CPU 코어로 클러스터를 구성해 보겠습니다. ml.p3.2xlarge에는 8개의 CPU 코어가 있고 ml.c5.4xlarge에는 16개의 CPU 코어가 있으므로 1개의 ml.p3.2xlarge 인스턴스와 2개의 ml.c5.4xlarge 인스턴스를 사용하여 이 작업을 수행할 수 있습니다.

In [ ]:
%%time

# Build CPU image
cpu_repository_short_name = "sagemaker-roboschool-ray-%s" % "cpu"
docker_build_args = {
    'CPU_OR_GPU': "cpu", 
    'AWS_REGION': boto3.Session().region_name,
}
cpu_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
print("Using CPU ECR image %s" % cpu_image_name)

# Build GPU image
gpu_repository_short_name = "sagemaker-roboschool-ray-%s" % "gpu"
docker_build_args = {
    'CPU_OR_GPU': "gpu", 
    'AWS_REGION': boto3.Session().region_name,
}
gpu_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
print("Using GPU ECR image %s" % gpu_image_name)

In [ ]:
primary_cluster_instance_type = "ml.p3.2xlarge"
primary_cluster_instance_count = 1

secondary_cluster_instance_type = "ml.c5.4xlarge"
secondary_cluster_instance_count = 2

total_cpus = 40 - 1 # Leave one for ray scheduler
total_gpus = 1

In [ ]:
primary_cluster_instance_type = "ml.p3.16xlarge"
primary_cluster_instance_count = 1

secondary_cluster_instance_type = "ml.c5.4xlarge"
secondary_cluster_instance_count = 2

total_cpus = 40 - 1 # Leave one for ray scheduler
total_gpus = 8

다음으로, 훈련하려는 roboschool 에이전트를 선택합니다. 이기종(heterogeneous) 훈련의 경우 인스턴스 간 동기화를 지원하는 몇 가지 추가 파라메터들을 훈련 작업에 전달합니다.

- s3_bucket, s3_prefix: 마스터 IP 주소와 같은 메타데이터 저장에 사용
- rl_cluster_type: "기본" 또는 "보조"
- aws_region: VPC 모드에서 S3에 연결하는 데 필요
- rl_num_instances_secondary: 보조 클러스터의 노드 수
- subnets, security_group_ids: VPC 모드에 필요

In [ ]:
roboschool_problem = 'reacher'
job_name_prefix = 'rl-roboschool-distributed-'+ roboschool_problem

s3_output_path = 's3://{}/'.format(s3_bucket) # SDK appends the job name and output folder

# We explicitly need to specify these params so that the two jobs can synchronize using the metadata stored here
s3_bucket = sage_session.default_bucket()
s3_prefix = "dist-ray-%s-1GPU-40CPUs" % (roboschool_problem)

# Make sure that the prefix is empty
!aws s3 rm --recursive s3://{s3_bucket}/{s3_prefix}    

### 기본 클러스터 시작 (1 GPU 훈련 인스턴스)

In [ ]:
primary_cluster_estimator = RLEstimator(entry_point="train-%s.py" % roboschool_problem,
                            source_dir='src',
                            dependencies=["common/sagemaker_rl"],
                            image_name=gpu_image_name,
                            role=role,
                            train_instance_type=primary_cluster_instance_type,
                            train_instance_count=primary_cluster_instance_count,
                            output_path=s3_output_path,
                            base_job_name=job_name_prefix,
                            metric_definitions=metric_definitions,
                            train_max_run=int(3600 * .5), # Maximum runtime in seconds
                            hyperparameters={
                                "s3_prefix": s3_prefix, # Important for syncing
                                "s3_bucket": s3_bucket, # Important for syncing
                                "aws_region": boto3.Session().region_name, # Important for S3 connection
                                "rl_cluster_type": "primary", # Important for syncing
                                "rl_num_instances_secondary": secondary_cluster_instance_count, # Important for syncing
                                "rl.training.config.num_workers": total_cpus,
                                "rl.training.config.train_batch_size": 20000,
                                "rl.training.config.num_gpus": total_gpus,
                            },
                            subnets=default_subnets, # Required for VPC mode
                            security_group_ids=default_security_groups # Required for VPC mode
                        )

primary_cluster_estimator.fit(wait=False)
primary_job_name = primary_cluster_estimator.latest_training_job.job_name
print("Primary Training job: %s" % primary_job_name)

### 보조 클러스터 시작 (2 CPU 인스턴스)

In [ ]:
secondary_cluster_estimator = RLEstimator(entry_point="train-%s.py" % roboschool_problem,
                            source_dir='src',
                            dependencies=["common/sagemaker_rl"],
                            image_name=cpu_image_name,
                            role=role,
                            train_instance_type=secondary_cluster_instance_type,
                            train_instance_count=secondary_cluster_instance_count,
                            output_path=s3_output_path,
                            base_job_name=job_name_prefix,
                            metric_definitions=metric_definitions,
                            train_max_run=3600, # Maximum runtime in seconds
                            hyperparameters={
                                "s3_prefix": s3_prefix, # Important for syncing
                                "s3_bucket": s3_bucket, # Important for syncing
                                "aws_region": boto3.Session().region_name, # Important for S3 connection
                                "rl_cluster_type": "secondary", # Important for syncing
                            },
                            subnets=default_subnets, # Required for VPC mode
                            security_group_ids=default_security_groups # Required for VPC mode
                        )

secondary_cluster_estimator.fit(wait=False)
secondary_job_name = secondary_cluster_estimator.latest_training_job.job_name
print("Secondary Training job: %s" % secondary_job_name)

### 시각화

In [ ]:
print("Job name: {}".format(primary_job_name))

s3_url = "s3://{}/{}".format(s3_bucket,primary_job_name)

if local_mode:
    output_tar_key = "{}/output.tar.gz".format(primary_job_name)
else:
    output_tar_key = "{}/output/output.tar.gz".format(primary_job_name)

intermediate_folder_key = "{}/output/intermediate/".format(primary_job_name)
output_url = "s3://{}/{}".format(s3_bucket, output_tar_key)
intermediate_url = "s3://{}/{}".format(s3_bucket, intermediate_folder_key)

print("S3 job path: {}".format(s3_url))
print("Output.tar.gz location: {}".format(output_url))
print("Intermediate folder path: {}".format(intermediate_url))
    
tmp_dir = "/tmp/{}".format(primary_job_name)
os.system("mkdir {}".format(tmp_dir))
print("Create local folder {}".format(tmp_dir))

### 훈련 롤아웃 비디오 가져오기

특정 롤아웃의 비디오는 훈련 중 S3에 기록됩니다. 여기에서는 S3에서 마지막 10개의 비디오 클립을 가져 와서 마지막 비디오를 렌더링합니다.

In [ ]:
recent_videos = wait_for_s3_object(s3_bucket, intermediate_folder_key, tmp_dir, 
                                fetch_only=(lambda obj: obj.key.endswith(".mp4") and obj.size>0), limit=10)

In [ ]:
last_video = sorted(recent_videos)[-1]  # Pick which video to watch
os.system("mkdir -p ./src/tmp_render_heterogeneous/ && cp {} ./src/tmp_render_heterogeneous/last_video.mp4".format(last_video))
HTML('<video src="./src/tmp_render_heterogeneous/last_video.mp4" controls autoplay></video>')

### 훈련 작업에 대한 지표 plot

CloudWatch 지표에 기록된 알고리즘 지표를 사용하여 실행 중인 훈련의 보상 지표를 볼 수 있습니다. 시간이 지남에 따라, 모델의 성능을 볼 수 있도록 이를 plot할 수 있습니다.

In [ ]:
%matplotlib inline
from sagemaker.analytics import TrainingJobAnalytics

df = TrainingJobAnalytics(primary_job_name, ['episode_reward_mean']).dataframe()
num_metrics = len(df)
if num_metrics == 0:
    print("No algorithm metrics found in CloudWatch")
else:
    plt = df.plot(x='timestamp', y='value', figsize=(12,5), legend=True, style='b-')
    plt.set_ylabel('Mean reward per episode')
    plt.set_xlabel('Training time (s)')

위의 시각화 셀을 반복해서 실행하여 최신 비디오를 얻거나, 훈련 작업이 진행됨에 따라 최신 지표를 볼 수 있습니다.